In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import networkx as nx
import os
import sys
sys.path.append('..')
from py_files.michele_algo import *

# Import Data

In [ ]:
def inspect_network(folder):
    print(folder)
    creation_date = str(folder.split('/')[-3])
    df = pd.DataFrame(columns = ['name', 'scrape_date', '#nodes', '#edges', 'average_shortest_path_length', 'Generalized_euclidean', 'Our_polarity_score'])
    for file in os.listdir(folder):
        graph = nx.read_gexf(folder+file)
        graph = nx.to_undirected(graph)
        S = [graph.subgraph(c).copy() for c in sorted(nx.connected_components(graph), key=len, reverse=True)]
        graph = S[0]
        name = file[:-5]

        attr_dict = {
            'name' : name,
            'scrape_date' : creation_date, 
            '#nodes' : len(graph.nodes()),
            '#edges' : len(graph.edges()),
            'average_shortest_path_length' : nx.average_shortest_path_length(graph),
            'Generalized_euclidean' : do_ge(graph),
            'Our_polarity_score' : driver_g(graph) 
        }
        df = df.append(attr_dict, ignore_index = True)
    return df

In [ ]:
main_dir = '../data/date_folders/'
First = True
for folder in os.listdir(main_dir):
    print(main_dir+folder+'/graphs/')
    if First:
        df = inspect_network(main_dir+folder+'/graphs/')
        First = False
    else:
        try:
            df = inspect_network(main_dir+folder+'/graphs/').append(df)
        except:
            pass

In [ ]:
#df = df.set_index('name')

In [ ]:
df

In [ ]:
sns.pairplot(df, 
             #hue='name'
             )

# Barplots of main measures

In [ ]:
# general seaborn settings
import matplotlib.pyplot as plt

sns.set_style('darkgrid')
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})

In [ ]:
# generalized euclidean (michele measure 1)
df_GE = df.copy().sort_values('Generalized_euclidean')
df_GE = df_GE[df_GE['name'].isin([
    'PoliticalDiscussion',
    'Republican',
    'Cooking',
    'democrats',
    'ukpolitics',
    'politics',
    'worldnews',
    'News',
    'antiwork'
])]
bar_GE = sns.barplot(y=df_GE['name'], x=df_GE['Generalized_euclidean'], color='tab:blue')
bar_GE.axes.set_title('Ideological Distance of Subreddits',fontsize=20)
bar_GE.set_xlabel('Ideological Distance',fontsize=15)
bar_GE.set_ylabel('Subreddit',fontsize=15)
bar_GE.tick_params(labelsize=10)
fig = bar_GE.get_figure()
fig.savefig('../data/plots/bar_GE.png', bbox_inches='tight')

In [ ]:
# Correlation between ideological distance and comment sentiment
df_correlation = df_GE.copy().sort_values('Our_polarity_score')
bar_correlation = sns.barplot(x=df_correlation['Our_polarity_score'], y=df_correlation['name'], color='tab:blue')
bar_correlation.set(xlim=(-0.4,0.4))
bar_correlation.axes.set_title('Relational Component Score between\nIdeological Distance and Comment Sentiment',fontsize=20)
bar_correlation.set_xlabel('Relational Component Score',fontsize=15)
bar_correlation.set_ylabel('Subreddit',fontsize=15)
bar_correlation.tick_params(labelsize=10)
fig = bar_correlation.get_figure()
fig.savefig('../data/plots/bar_correlation.png', bbox_inches='tight')

### New plot

In [ ]:
df_GE.rename(columns={'name':'Subreddit'}, inplace = True)
scatter = sns.scatterplot(data=df_GE, x='Generalized_euclidean', y='Our_polarity_score', hue='Subreddit')
plt.axhline(y=0, color='black', ls='dashed')
sns.move_legend(scatter, 'upper left', bbox_to_anchor=(1, 1), frameon=False)
# plt.axvline(x=60, color='black', ls='dashed')
scatter.set(xlim=(0,100), ylim=(-0.4,0.4))
scatter.axes.set_title('Ideological Distance and Relational Component Score',fontsize=20)
scatter.set_xlabel('Ideological Distance',fontsize=15)
scatter.set_ylabel('Relational Component Score',fontsize=15)
scatter.tick_params(labelsize=10)
fig = scatter.get_figure()
fig.savefig('../data/plots/scatter.png', bbox_inches='tight')